# Predictions and Performance Evaluation

Welcome to this notebook. Here, we aim to achieve two main tasks:

1. **Predictions for 2024 and 2025:** In the first part of the notebook, we will train a model using available data and generate predictions for the years 2024 and 2025. The code for this task starts right after this introductory note.

2. **Performance Evaluation for Predictions:** Following the prediction generation, we will drop the data for 2019 from our dataset and use our trained model to predict data for this year. We will then compare these predictions with the actual data from 2019 to evaluate the model's performance.

**Please note:** It's crucial to **restart the kernel** after the initial prediction task (2024 and 2025) and before proceeding with the performance evaluation. Restarting the kernel ensures that the environment is clean and that previously defined variables or functions do not interfere with the evaluation process.

Let's begin!

In [1]:
# Import the modules
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
import numpy as np
import sqlite3
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Connect to SQLite database
# Create a connection to the SQLite database to load the data and check if the data has been stored correctly
conn = sqlite3.connect('Resources/immigration_selected_2005_2019_sqlite.sqlite')

# Write a SQL query to load the data from the table in the SQLite database
df = pd.read_sql_query("SELECT * from immigration_selected_2005_2019_sqlite", conn)
df.head()

,Year,Region and country of birth,Total Permanent Residents,Percentage,Alabama,Alaska,Arizona,Arkansas,California,Colorado,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,2005,Total,1122373,100.00,4200,1525,18988,2698,232023,11977,...,881,8962,95958,5082,1042,27100,26482,847,7909,321
1,2005,"China, People's Republic",69967,6.23,328,92,543,202,17668,765,...,51,637,4139,217,50,1327,1508,101,593,28
2,2005,Dominican Republic,27504,2.45,5,42,22,0,82,6,...,0,22,119,11,0,90,18,6,39,0
3,2005,India,84681,7.54,431,15,739,215,14724,516,...,23,900,7139,147,74,2776,1747,133,876,0
4,2005,Iran,13887,1.24,48,4,285,9,7059,131,...,0,150,1002,135,4,562,318,18,48,0


In [3]:
# Drop the "Percentage" column
df = df.drop('Percentage', axis=1)

In [4]:
df

,Year,Region and country of birth,Total Permanent Residents,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,2005,Total,1122373,4200,1525,18988,2698,232023,11977,15335,...,881,8962,95958,5082,1042,27100,26482,847,7909,321
1,2005,"China, People's Republic",69967,328,92,543,202,17668,765,894,...,51,637,4139,217,50,1327,1508,101,593,28
2,2005,Dominican Republic,27504,5,42,22,0,82,6,319,...,0,22,119,11,0,90,18,6,39,0
3,2005,India,84681,431,15,739,215,14724,516,1571,...,23,900,7139,147,74,2776,1747,133,876,0
4,2005,Iran,13887,48,4,285,9,7059,131,88,...,0,150,1002,135,4,562,318,18,48,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2019,Iran,6640,16,0,92,26,2693,70,41,...,3,46,701,40,3,214,258,5,54,12
131,2019,Mexico,156052,507,77,8520,891,47725,4243,259,...,74,1286,38736,2034,8,796,3266,44,1418,113
132,2019,Pakistan,13921,44,0,67,28,1706,60,257,...,8,67,2095,72,0,943,286,23,126,0
133,2019,Philippines,45920,235,514,916,180,14934,387,209,...,47,672,2629,202,23,1019,1492,70,252,31


In [5]:
# List of unique countries
countries = df['Region and country of birth'].unique()
countries

array(['Total', "China, People's Republic", 'Dominican Republic', 'India',
       'Iran', 'Mexico', 'Pakistan', 'Philippines', 'United Kingdom'],
      dtype=object)

In [6]:
# Drop 'Total' from the unique countries
countries = np.delete(countries, np.where(countries == 'Total'))
countries

array(["China, People's Republic", 'Dominican Republic', 'India', 'Iran',
       'Mexico', 'Pakistan', 'Philippines', 'United Kingdom'],
      dtype=object)

In [7]:
# DataFrame to store predictions
df_predictions = pd.DataFrame()
df_scores = pd.DataFrame()

In [8]:
# List of all states
states = df.columns[3:]

In [9]:
# Collect all predictions and true values for final R-squared calculation
all_preds = []
all_true = []

In [10]:
for country in countries:
    df_country = df[df['Region and country of birth'] == country]
    for state in states:
        X = df_country[['Year']]
        y = df_country[state].values
        
        scaler = MinMaxScaler()
        X = scaler.fit_transform(X)

        # We need to transform our time series data into a supervised learning problem
        # Let's assume we're using a lag of 1 (t-1 to predict t)
        X_new = np.array([y[i-1] for i in range(1, len(y))]).reshape(-1, 1)
        y_new = np.array([y[i] for i in range(1, len(y))])
        X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

        # Create and train the model
        model = RandomForestRegressor()
        model.fit(X_train, y_train)

        # Make prediction 
        y_pred = model.predict(X_test)
        
        # Append predicted and true values to the respective lists
        all_preds.extend(y_pred)
        all_true.extend(y_test)

        # Calculate RMSE, and MAPE
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-10))) * 100
        
        # Calculate MAE and MSE
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        
        # Calculate CVRMSE
        cvrmse = rmse / np.mean(y_test) * 100
        
        # Store scores in the scores DataFrame
        df_scores = df_scores.append({
            'Country': country,
            'State': state,
            'RMSE': rmse,
            'MAPE%': mape,
            'MAE': mae,
            'MSE': mse,
            'CVRMSE%': cvrmse
        }, ignore_index=True)

        # Predict for 2024 and 2025
        next_years = np.array([y[-1], model.predict(y[-1].reshape(-1, 1))]).reshape(-1, 1)
        predictions = model.predict(next_years)
        
        for i, year in enumerate([2020, 2025]):
            df_predictions = df_predictions.append({
                'Year': year,
                'Country': country,
                'State': state,
                'Predicted Immigration': predictions[i]
            }, ignore_index=True)

C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  next_years = np.array([y[-1], model.predict(y[-1].reshape(-1, 1))]).reshape(-1, 1)
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method 

C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  next_years = np.array([y[-1], model.predict(y[-1].reshape(-1, 1))]).reshape(-1, 1)
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method 

C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  next_years = np.array([y[-1], model.predict(y[-1].reshape(-1, 1))]).reshape(-1, 1)
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method 

C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  next_years = np.array([y[-1], model.predict(y[-1].reshape(-1, 1))]).reshape(-1, 1)
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method 

C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  next_years = np.array([y[-1], model.predict(y[-1].reshape(-1, 1))]).reshape(-1, 1)
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method 

...

C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  next_years = np.array([y[-1], model.predict(y[-1].reshape(-1, 1))]).reshape(-1, 1)
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method 

C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  next_years = np.array([y[-1], model.predict(y[-1].reshape(-1, 1))]).reshape(-1, 1)
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method 

C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  next_years = np.array([y[-1], model.predict(y[-1].reshape(-1, 1))]).reshape(-1, 1)
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method 

C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  next_years = np.array([y[-1], model.predict(y[-1].reshape(-1, 1))]).reshape(-1, 1)
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method 

C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  next_years = np.array([y[-1], model.predict(y[-1].reshape(-1, 1))]).reshape(-1, 1)
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_29796\367611020.py:54: FutureWarning: The frame.append method 

In [11]:
# Calculate overall R2 score
overall_r2 = r2_score(all_true, all_preds)
print(f'Overall R2 score: {overall_r2}')

Overall R2 score: 0.9707978087108843


In [12]:
# Overall RMSE
overall_rmse = np.sqrt(mean_squared_error(all_true, all_preds))
print('Overall RMSE: ', overall_rmse)

Overall RMSE:  729.1620583722561


In [13]:
# Save the scores DataFrame to a CSV file
df_scores.to_csv('Scores/immigration_scores2024-2025_RandomForestRegressor.csv', index=False)

In [14]:
# Pivot the DataFrame to have each state in a separate column
df_pivot = df_predictions.pivot_table(index=['Year', 'Country'], columns='State', values='Predicted Immigration')

In [15]:
# Add a new column 'Total PR' which is the sum of all states in each row
df_pivot['Total PR'] = df_pivot.sum(axis=1)

In [16]:
print(df_pivot.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 16 entries, (2020, "China, People's Republic") to (2025, 'United Kingdom')
Data columns (total 52 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Alabama               16 non-null     float64
 1   Alaska                16 non-null     float64
 2   Arizona               16 non-null     float64
 3   Arkansas              16 non-null     float64
 4   California            16 non-null     float64
 5   Colorado              16 non-null     float64
 6   Connecticut           16 non-null     float64
 7   Delaware              16 non-null     float64
 8   District of Columbia  16 non-null     float64
 9   Florida               16 non-null     float64
 10  Georgia               16 non-null     float64
 11  Hawaii                16 non-null     float64
 12  Idaho                 16 non-null     float64
 13  Illinois              16 non-null     float64
 14  Indiana              

In [17]:
# Find all 'State' columns and convert to 'int64'
for col in df_pivot.columns:
    if col not in ['Year', 'Country']:
        df_pivot[col] = df_pivot[col].astype('int64')

In [18]:
print(df_pivot.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 16 entries, (2020, "China, People's Republic") to (2025, 'United Kingdom')
Data columns (total 52 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   Alabama               16 non-null     int64
 1   Alaska                16 non-null     int64
 2   Arizona               16 non-null     int64
 3   Arkansas              16 non-null     int64
 4   California            16 non-null     int64
 5   Colorado              16 non-null     int64
 6   Connecticut           16 non-null     int64
 7   Delaware              16 non-null     int64
 8   District of Columbia  16 non-null     int64
 9   Florida               16 non-null     int64
 10  Georgia               16 non-null     int64
 11  Hawaii                16 non-null     int64
 12  Idaho                 16 non-null     int64
 13  Illinois              16 non-null     int64
 14  Indiana               16 non-null     int64
 15  Iowa

In [19]:
# Reset the index
df_pivot.reset_index(inplace=True)

In [20]:
# Save the predictions DataFrame to a CSV file
df_pivot.to_csv('Results/immigration_predictions2024-2025_RandomForestRegressor.csv', index=False)

# Performance Evaluation for Predictions

In this notebook, we initially use a dataset spanning from 2005 to 2019 to train our model and generate predictions. After this initial analysis, we further test our model's performance by using a slightly modified dataset.

We will follow the below steps:

1. Run the complete notebook with the original dataset (2005-2019).
2. After the first run is complete, we will modify the dataset by dropping the 2019 data. We will then use this updated dataset to predict data for the year 2019.
3. The predicted 2019 data will be compared with the actual 2019 data to evaluate the performance of our model.

This evaluation process allows us to better understand how our model performs when faced with unseen data and helps validate our model's generalization capabilities.

# Instructions for Running this Part of the Code

To ensure that this section of the notebook runs correctly, please restart the kernel and then run the cells from this point forward. This ensures that the environment is clean and free from any variables or functions that may have been defined in other parts of the notebook.

After you have restarted the kernel, you can proceed with running the following cells.

In [11]:
# Import the modules
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
import numpy as np
import sqlite3
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Connect to SQLite database
# Create a connection to the SQLite database to load the data and check if the data has been stored correctly
conn = sqlite3.connect('Resources/immigration_selected_2005_2019_sqlite.sqlite')

# Write a SQL query to load the data from the table in the SQLite database
df = pd.read_sql_query("SELECT * from immigration_selected_2005_2019_sqlite", conn)

In [3]:
# Drop the "Percentage" column
df = df.drop('Percentage', axis=1)

In [4]:
# Exclude the data of the year 2019
df = df[df['Year'] != 2019]
df

,Year,Region and country of birth,Total Permanent Residents,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,2005,Total,1122373,4200,1525,18988,2698,232023,11977,15335,...,881,8962,95958,5082,1042,27100,26482,847,7909,321
1,2005,"China, People's Republic",69967,328,92,543,202,17668,765,894,...,51,637,4139,217,50,1327,1508,101,593,28
2,2005,Dominican Republic,27504,5,42,22,0,82,6,319,...,0,22,119,11,0,90,18,6,39,0
3,2005,India,84681,431,15,739,215,14724,516,1571,...,23,900,7139,147,74,2776,1747,133,876,0
4,2005,Iran,13887,48,4,285,9,7059,131,88,...,0,150,1002,135,4,562,318,18,48,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,2018,Iran,10116,26,0,155,11,4527,94,60,...,0,68,960,51,3,407,403,7,40,4
122,2018,Mexico,161858,618,86,8025,901,53572,4053,283,...,76,1293,36535,1564,0,840,3413,31,1536,114
123,2018,Pakistan,15802,78,3,119,46,2003,78,256,...,6,93,2032,56,0,1238,226,15,138,4
124,2018,Philippines,47258,213,511,657,179,15412,350,215,...,64,573,2583,192,26,1109,1553,89,221,57


In [5]:
# List of unique countries
countries = df['Region and country of birth'].unique()
countries

array(['Total', "China, People's Republic", 'Dominican Republic', 'India',
       'Iran', 'Mexico', 'Pakistan', 'Philippines', 'United Kingdom'],
      dtype=object)

In [6]:
# Drop 'Total' from the unique countries
countries = np.delete(countries, np.where(countries == 'Total'))
countries

array(["China, People's Republic", 'Dominican Republic', 'India', 'Iran',
       'Mexico', 'Pakistan', 'Philippines', 'United Kingdom'],
      dtype=object)

In [7]:
# DataFrame to store predictions
df_predictions = pd.DataFrame()
df_scores = pd.DataFrame()

In [8]:
# List of all states
states = df.columns[3:]

In [9]:
# Collect all predictions and true values for final R-squared calculation
all_preds = []
all_true = []

In [12]:
for country in countries:
    df_country = df[df['Region and country of birth'] == country]
    for state in states:
        X = df_country[['Year']]
        y = df_country[state].values
        
        scaler = MinMaxScaler()
        X = scaler.fit_transform(X)

        # We need to transform our time series data into a supervised learning problem
        # Let's assume we're using a lag of 1 (t-1 to predict t)
        X_new = np.array([y[i-1] for i in range(1, len(y))]).reshape(-1, 1)
        y_new = np.array([y[i] for i in range(1, len(y))])
        X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

        # Create and train the model
        model = RandomForestRegressor()
        model.fit(X_train, y_train)

        # Make prediction 
        y_pred = model.predict(X_test)
        
        # Append predicted and true values to the respective lists
        all_preds.extend(y_pred)
        all_true.extend(y_test)

        # Calculate RMSE, and MAPE
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-10))) * 100
        
        # Calculate MAE and MSE
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        
        # Calculate CVRMSE
        cvrmse = rmse / np.mean(y_test) * 100
        
        # Store scores in the scores DataFrame
        df_scores = df_scores.append({
            'Country': country,
            'State': state,
            'RMSE': rmse,
            'MAPE%': mape,
            'MAE': mae,
            'MSE': mse,
            'CVRMSE%': cvrmse
        }, ignore_index=True)

        # Predict for 2019
        last_year = y[-1].reshape(-1, 1)  # The last year in the filtered data
        prediction = model.predict(last_year)
        
        df_predictions = df_predictions.append({
            'Year': 2019,
            'Country': country,
            'State': state,
            'Predicted Immigration': prediction[0]
        }, ignore_index=True)

C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_

C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_

C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_

C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_

C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_

...

C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_

C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_

C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_

C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_

C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_19252\3902589462.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\ehsan\AppData\Local\Temp\ipykernel_

In [13]:
# Calculate overall R2 score
overall_r2 = r2_score(all_true, all_preds)
print(f'Overall R2 score: {overall_r2}')

Overall R2 score: 0.9737232162229371


In [19]:
# Save the scores DataFrame to a CSV file
df_scores.to_csv('Scores/immigration_scores2019_RandomForestRegressor.csv', index=False)

In [20]:
# Pivot the DataFrame to have each state in a separate column
df_pivot = df_predictions.pivot_table(index=['Year', 'Country'], columns='State', values='Predicted Immigration')

In [21]:
# Add a new column 'Total PR' which is the sum of all states in each row
df_pivot['Total PR'] = df_pivot.sum(axis=1)

In [22]:
# Find all 'State' columns and convert to 'int64'
for col in df_pivot.columns:
    if col not in ['Year', 'Country']:
        df_pivot[col] = df_pivot[col].astype('int64')

In [23]:
# Reset the index
df_pivot.reset_index(inplace=True)

In [24]:
# Save the predictions DataFrame to a CSV file
df_pivot.to_csv('Results/immigration_predictions2019_RandomForestRegressor.csv', index=False)